# Create the features DF
* using by_postal_code dataset

# Import

In [1]:
import pandas as pd
import json
import numpy as np
from collections import Counter
import matplotlib.pyplot as plt
import pickle
import math 

from scipy import sparse as sp
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import LabelEncoder
from sklearn.feature_selection import chi2
from sklearn.feature_selection import SelectKBest

In [3]:
df = pd.read_pickle ('/home/adam/Steph_C/my_thesis/data/ORI_by_postal_code.pkl').reset_index(drop=True)
# LDA_df = pd.read_pickle ('../data/LDA_training.pkl')

In [4]:
df.shape

(209052, 18)

In [5]:
# LDA_df.shape

In [6]:
df.head()

,business_id,stars_x,useful,funny,cool,text,date,name,address,city,state,postal_code,stars_y,review_count,is_open,attributes,categories,hours
0,HTqXI5S2XcSlh_ylx9sE6g,5.0,1,1,1,I've only had the cannolis here but they are a...,2018-03-23 14:35:33,Termini Bros Bakery,12th St & Filbert St,Philadelphia,PA,19107,4.0,263,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Restaurants, Bakeries","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
1,HTqXI5S2XcSlh_ylx9sE6g,5.0,0,0,0,It's all about the cannoli! Don't try to share...,2018-02-11 20:44:13,Termini Bros Bakery,12th St & Filbert St,Philadelphia,PA,19107,4.0,263,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Restaurants, Bakeries","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
2,HTqXI5S2XcSlh_ylx9sE6g,4.0,0,0,0,Good tasting food that's a staple of the Readi...,2018-03-06 02:20:11,Termini Bros Bakery,12th St & Filbert St,Philadelphia,PA,19107,4.0,263,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Restaurants, Bakeries","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
3,HTqXI5S2XcSlh_ylx9sE6g,4.0,0,0,0,I really enjoyed the cannoli from here. I did ...,2018-03-29 19:21:09,Termini Bros Bakery,12th St & Filbert St,Philadelphia,PA,19107,4.0,263,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Restaurants, Bakeries","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."
4,HTqXI5S2XcSlh_ylx9sE6g,2.0,1,0,0,One day their bakery was wonderful. The next ...,2018-05-13 01:21:05,Termini Bros Bakery,12th St & Filbert St,Philadelphia,PA,19107,4.0,263,1,"{'RestaurantsPriceRange2': '2', 'BusinessParki...","Food, Restaurants, Bakeries","{'Monday': '10:0-18:0', 'Tuesday': '10:0-18:0'..."


# Preprocessing 

In [ ]:
from nltk.stem import PorterStemmer
import nltk
from nltk.corpus import stopwords

In [ ]:
stop_words = stopwords.words('english')

def preprocess(text : str) -> list:
    
    tokens = text.lower().rstrip(' ').rstrip('\n').rstrip('.').split(' ')[:3000]
    
    p = PorterStemmer()
    stemmed_doc =[]
    for token in tokens:
        stemmed_doc.append(p.stem(token))
        
    tokens = [i.split(' ') for i in stemmed_doc]
    
    
    final_tokens =[]


    for l in tokens:
        for word in l:
            word = word.rstrip('.').rstrip('\'').rstrip(',')
            if word not in final_tokens and word not in stop_words and len(word)>0:
                final_tokens.append(word)
    return final_tokens

# Review-based Market Attractiveness Features (MAF)
* Turn every restaurant and city to a doc of reviews
* Use `dict` to store the dic index of restaurant and city
* review - LDA
* location - LDA 

In [ ]:
from pprint import pprint
import gensim
import gensim.corpora as corpora
from gensim.utils import simple_preprocess
from nltk.corpus import stopwords
from gensim.models import CoherenceModel
import pyLDAvis
import pyLDAvis.gensim_models
import matplotlib.pyplot as plt
import nltk

In [ ]:
# # aspect creation 
# total_df = pd.concat([df,LDA_df] , ignore_index=True)

# docs = []
# index =0 


# # restaurant
# for restaurant in Counter(df.name):
#     if index%10 ==0:
#         print(f'Now {index} Restaurant ... {restaurant}')
#     tmp = df[df.name == restaurant]
#     texts = ''.join(tmp.text.tolist())
#     docs.append(preprocess(texts))
#     indxe +=1

# # city 
# for city_name in Counter(df.city):
#     if index%5 ==0:
#         print(f'Now {index} City ... {city_name}')
#     tmp = total_df[total_df.city == city_name]
#     texts = ''.join(tmp.text.tolist())
#     docs.append(preprocess(texts))
#     index +=1   

# # save pickle
# with open('./data/docs.pkl', 'wb') as f:
#     pickle.dump(docs, f)

In [ ]:
with open('../data/docs.pkl', 'rb') as f:
    docs = pickle.load(f)
    

## LDA

In [ ]:
index = 0
name_city_dict = {}
for restaurant in Counter(df.name):
    name_city_dict[restaurant]=index
    index +=1
    
for city_name in Counter(df.city):
    name_city_dict[city_name]=index
    index +=1   

In [ ]:
### LDA ###
common_dictionary = corpora.Dictionary(docs)
common_corpus = [common_dictionary.doc2bow(text) for text in docs]


# Train the model on the corpus.
lda =  gensim.models.LdaMulticore(corpus = common_corpus, id2word= common_dictionary, num_topics= 10)
lda.show_topics()


In [ ]:
# check the topic coherence
coherence_model_lda = CoherenceModel( model=lda , texts=docs , dictionary=common_dictionary , coherence='c_v' )
conherence_lda = coherence_model_lda.get_coherence()
print(conherence_lda)

In [ ]:
# create restaurant aspects

rest_aspect = {}
loca_aspect = {}

for name in Counter(df.name):
    tmp_arr = np.zeros(10)
    for i in lda[common_corpus[name_city_dict[name]]]:
        tmp_arr[i[0]]=i[1]
    rest_aspect[name]= tmp_arr

for location in Counter(df.city):
    tmp_arr = np.zeros(10)
    for i in lda[common_corpus[name_city_dict[location]]]:
        tmp_arr[i[0]]=i[1]
    loca_aspect[location]= tmp_arr

## Affinity & Complementary

In [ ]:
df = df.reset_index()

In [ ]:
df['affinity']=""
df['complementary']=""

for i in range(len(df)):
    df.affinity[i] = rest_aspect[df.name[i]]*loca_aspect[df.city[i]]
    df.complementary[i] = rest_aspect[df.name[i]]*(1-loca_aspect[df.city[i]]) 

In [ ]:
df.head()

# Review-based Market Competitiveness Feature (MCF)
* review - LDA rank 

# Geographic Features (GeoF)
* Density : the restaurant number in the same postal code
* Neighbourhood Entropy : entropy measure of the frequency of restaurant categories
    * A location with **higher entropy value** is expected to be **more diverse** in terms of restaurant categories,
* Competitiveness : the proportion of nearby restaurants with the same type of restaurant 
* Jensen Quality : Jensen Quality of each venue category as a feature value, rather than simply aggregating Jensen Quality values over all venue categories as a single feature value. 

In [7]:
df.columns

Index(['business_id', 'stars_x', 'useful', 'funny', 'cool', 'text', 'date',
       'name', 'address', 'city', 'state', 'postal_code', 'stars_y',
       'review_count', 'is_open', 'attributes', 'categories', 'hours'],
      dtype='object')

In [8]:
# create dataset without review
# unhash if neccessary
import math

# drop duplicates
df = df.drop_duplicates(subset=['name','postal_code']).reset_index(drop=True)

# calculate the density by Postal code
df['density']=""
df['entropy']=""
df['competitiveness']=""


for i in range(len(df)):
    
    place = df[df.postal_code == df.postal_code[i]]
    
    # density 
    df.density[i] = len(Counter(place.name))
    
    # entropy
    entropy_sum = 0
    
    for category in Counter(place.categories):
        entropy_sum+=(len(Counter(place[place.categories==category].name))/len(Counter(place.name)))*np.log(len(Counter(place[place.categories==category].name))/len(Counter(place.name)))
    df.entropy[i] = -entropy_sum
    
    # Competitiveness
    comp = -(len(Counter(place[place.categories == df.categories[i]].name))/len(Counter(place.name)))
    df.competitiveness[i]= comp

df.to_pickle("../data/FE_by_postoal_code_without_review.pkl")    


/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:19: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:26: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
/home/adam/.local/lib/python3.7/site-packages/ipykernel_launcher.py:30: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy


In [ ]:
df.to_pickle("../data/FE_by_postoal_code.pkl")    